In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from skimage import feature
from imutils import build_montages
from imutils import paths
import numpy as np
import cv2
import os
import pickle #importing the pickle file

def quantify_image(image):
    features = feature.hog(image, orientations=9, pixels_per_cell=(10, 10), cells_per_block=(2, 2), transform_sqrt=2, block_norm="L1")
    return features

# Function to load split data
def load_split(path):
               
    imagePaths = list(paths.list_images(path))
    
    # Initialize lists to store images and labels
    data = []
    labels = []
    
    # Iterate over the image paths
    for imagePath in imagePaths:
        label = imagePath.split(os.path.sep)[-2]
        
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (200, 200))
        image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        
        features = quantify_image(image)
        data.append(features)
        labels.append(label)
    
    # Return the loaded data as NumPy arrays
    return np.array(data), np.array(labels)

# Path to the training and testing directories
trainingPath = 'C:/Users/Nithish/Downloads/spiral (1)/spiral/training'
testingPath = 'C:/Users/Nithish/Downloads/spiral (1)/spiral/testing'

print("[INFO] Loading data...")
X_train, y_train = load_split(trainingPath)
X_test, y_test = load_split(testingPath)

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
print(X_train.shape, y_train.shape)

# Create a RandomForestClassifier and fit the training data
#clf = RandomForestClassifier()
#clf.fit(X_train, y_train)




[INFO] Loading data...
(72, 12996) (72,)


In [ ]:
print (" [INFO] training model")
model = RandomForestClassifier(n_estimators=100)
model.fit (X_train,y_train)

testingPaths = list(paths.list_images (testingPath))

idxs = np.arange(0, len (testingPaths)) 
idxs = np.random.choice (idxs, size=(25,), replace=False) 
images = []

for i in idxs:
    # Load the testing image, clone it, and resize it
    image = cv2.imread(testingPaths[i])
    output = image.copy()
    output = cv2.resize(output, (128, 128))

    # Pre-process the image
    image= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (200, 200))
    image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    features = quantify_image(image)
    preds = model.predict([features])
    label = le.inverse_transform(preds)[0]
    
    color = (0, 255, 0) if label=="healthy" else (0, 0, 255)
    cv2.putText(output, label, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    images.append(output)
    
    montage = build_montages(images, (128, 128), (5, 5))[0]
    cv2.imshow('Output', montage)
    cv2.waitKey(0)

    # Extract the label from the file name
    #label = testingPaths[i].split(os.path.sep)[-2]

    # Draw the colored class label on the output image
    #color = (0, 255, 0)  # Green color for "healthy" label
    #if label != "healthy":
        #color = (0, 0, 255)  # Red color for non-healthy labels

    

# Evaluate the classifier on the test data
#accuracy = model.score(X_test, y_test)
#print("Accuracy:", accuracy)

# Use the classifier to make predictions
#predictions = model.predict(X_test)
#print("Predictions:", predictions)

 [INFO] training model


In [ ]:
predictions = model.predict(X_test)
cm = confusion_matrix(y_test,predictions).flatten()
print(cm)
(tn, fp, fn, tp) = cm
accuracy = (tp + tn) / float(cm.sum())
print(accuracy)
      
model_path = 'C:/Users/Nithish/Downloads/spiral (1)/spiral.pkl'

# Save the model using pickle
with open(model_path, 'wb') as file:
    pickle.dump(model, file)

In [ ]:
import os
from skimage import feature

# Function to quantify the image using histogram-based features
def quantify_image(image):
    # Compute histogram of oriented gradients (HOG) features
    features = feature.hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
    return features

# Path to the training and testing directories
trainingPath = 'C:/Users/Nithish/Downloads/spiral (1)/spiral/training'
testingPath = 'C:/Users/Nithish/Downloads/spiral (1)/spiral/testing'

# Get the list of image paths in the training directory
trainingImagePaths = list(paths.list_images(trainingPath))

# Initialize lists to store images and labels
data = []
labels = []

# Iterate over the training image paths
for imagePath in trainingImagePaths:
    # Load the image, convert it to grayscale, and resize it
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (200, 200))

    # Quantify the image
    features = quantify_image(image)

    # Extract the label from the file name
    label = imagePath.split(os.path.sep)[-2]

    # Append the features and label to the lists
    data.append(features)
    labels.append(label)

In [11]:
images = []

for i in idxs:
    # Load the testing image, clone it, and resize it
    image = cv2.imread(testingPaths[i])
    output = image.copy()
    output = cv2.resize(output, (128, 128))

    # Pre-process the image
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_gray = cv2.resize(image_gray, (200, 200))
    _, image_thresh = cv2.threshold(image_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    # Append the processed image to the list
    images.append(output)

    # Extract the label from the file name
    label = testingPaths[i].split(os.path.sep)[-2]

    # Draw the colored class label on the output image
    color = (0, 255, 0)  # Green color for "healthy" label
    if label != "healthy":
        color = (0, 0, 255)  # Red color for non-healthy labels

    cv2.putText(output, label, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Create a montage using 128×128 "tiles" with 5 rows and 5 columns
montage = build_montages(images, (128, 128), (5, 5))[0]

# Show the output montage
cv2.imshow("Output", montage)
cv2.waitKey(0)

-1

In [23]:
 
    # make predictions on the testing data
predictions=model.predict(X_test)
print(predictions)

['healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'healthy' 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson'
 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson'
 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson'
 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson'
 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson'
 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson'
 'parkinson']


In [24]:
# compute the confusion matrix and and use it to derive the raw
# accuracy
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,predictions).flatten()
print(cm)
(tn, fp, fn, tp) = cm
accuracy = (tp + tn) / float(cm.sum())
print(accuracy)

[36  0  0 36]
1.0


In [25]:
import pickle


# Specify the file path to save the model
model_path = 'C:/Users/Nithish/Downloads/spiral (1)/spiral.pkl'

# Save the model using pickle
with open(model_path, 'wb') as file:
    pickle.dump(model, file)
